In [ ]:
%matplotlib inline
from sklearn.grid_search import GridSearchCV
import numpy as np
from scipy import io
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('ticks')
sns.set_context('notebook',font_scale=1.3)

### Load Data

In [ ]:
dat = io.loadmat('/data/task01_full.mat')
X, y, Xtest, ytest = dat['X'], dat['Y'].ravel(), dat['Xtest'], dat['Ytest'].ravel()

model = LogisticRegression(C=1.,penalty='l2')


### a) model selection

In [ ]:
def model_selection(model, param_grid):
    clf = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    clf.fit(X, y)
    return clf.best_estimator_
best = model_selection(model,  
                       {'C':10.**np.arange(-5.,4.5, .25), 'penalty':['l1','l2'], 'tol': [0.001, 0.0001, 0.00001]})
print(best)
yest = best.predict(Xtest)
print("Accuracy is %.2f" % (np.mean(yest == ytest),))


In [ ]:

#--------- Plotting --------------
mi, ma = X.ravel().min(), X.ravel().max()
ra = np.linspace(mi, ma, 30)
xp,yp = np.meshgrid(ra, ra)
Xp = np.c_[xp.ravel(),yp.ravel()]
Dp = best.decision_function(Xp).reshape(xp.shape)
Yp = best.predict(Xp)

with sns.axes_style('whitegrid'):
    fig, ax = plt.subplots(facecolor='w')
cb = ax.pcolormesh(xp,yp,Dp, cmap=plt.cm.gray,zorder=-10)
ax.contour(xp,yp,Dp, [0], zorder=-5)
ax.axis('tight')

idx = y == 1
ax.plot(X[idx,0],X[idx, 1], 'o', color='lime', zorder=10, label='train +')
ax.plot(X[~idx,0],X[~idx, 1], 'o', color='magenta', zorder=10, label='train -')

idx = ytest == 1
ax.plot(Xtest[idx,0],Xtest[idx, 1], 'o', color='dodgerblue', zorder=10, label='test +')
ax.plot(Xtest[~idx,0],Xtest[~idx, 1], 'o', color='gold', zorder=10, label='test -')

ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.legend()
ax.set_aspect(1)
fig.tight_layout()
fig.set_size_inches((7,7))

plt.show()
